# Machine learning process

- [Import data](#Import-data)
- [Test some algorithms](#Test-some-algorithms)
    - [First test - all values](#First-test---all-values)
    - [Removing outliers by distance using IQR](#Removing-gps-points-outliers--by-distance-using-IQR)
    - [Removing outliers by distance using Z-Score](#Removing-outliers-by-distance-using-Z-Score)
    - [Removing outliers by distance using Z-Score modified](#Removing-outliers-by-distance-using-Z-Score-modified)
- [GridSearchCV with Random Forest and Gradient Boost](#GridSearchCV-with-Random-Forest-and-Gradient-Boost)
    - [Random forest](#Random-forest)
    - [Gradient boost](#Gradient-boost)
- [Train all data with Random forest](#Train-all-data-with-Random-forest)
    - [Test (~2.56 km)](#Test)
- [Ploting results](#Ploting-results)

In [189]:
import pandas as pd
import numpy as np
import ast
import datetime
import holidays
import pickle
from math import radians, cos, sin, asin, sqrt

from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.svm import LinearSVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from xgboost import XGBRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

## Import data

In [2]:
def trata_dados(df):
    #--- Timestamp
    df['month'] = df['timestamp'].dt.month
    df['day'] = df['timestamp'].dt.day
    #df['cat_hour'] = pd.cut(df.timestamp.dt.hour, [-1, 8, 16, 23], labels=[0,1,2])
    df['hour'] = df.timestamp.dt.hour
    df['fds_fer'] = df.timestamp.dt.dayofweek
    df['fds_fer'] = df['fds_fer'].apply(lambda x: 1 if x==0 or x==6 else 0) #dom/sab = 0
    for data in sorted(holidays.Portugal(years=[2013, 2014])): # add holidays
        df['fds_fer'] = 1 if len(df[df.timestamp.dt.date==data])>0 else 0
    
    del df['timestamp']

    #--- Call_type hot encode
    df['call_type'] = preprocessing.LabelEncoder().fit_transform(df['call_type'])
    enc = OneHotEncoder().fit_transform(df[['call_type']])
    del df['call_type']
    df['call_type_a'] = enc.toarray()[:,0]
    df['call_type_b'] = enc.toarray()[:,1]
    df['call_type_c'] = enc.toarray()[:,2]

    #--- Fill na
    df.fillna(-1, inplace=True)

    return df

In [20]:
def carrega_dados(dataset='', n_rows=None, test_df=False):
    try:
        if test_df==False:
            usecols = ['CALL_TYPE', 'ORIGIN_CALL', 'ORIGIN_STAND', 'TAXI_ID', 'TIMESTAMP',
                       'dist_perc', 'start_lat', 'start_lon', 'stop_lat', 'stop_lon']
        else:
            usecols = ['TRIP_ID','CALL_TYPE', 'ORIGIN_CALL', 'ORIGIN_STAND', 'TAXI_ID', 'TIMESTAMP',
                       'dist_perc', 'start_lat', 'start_lon', 'stop_lat', 'stop_lon']

        if n_rows is None:
            df = pd.read_csv(dataset, usecols=usecols, parse_dates=True)
        else:
            df = pd.read_csv(dataset, usecols=usecols, parse_dates=True, nrows=n_rows)

        # header in lower case
        df.columns = df.columns.map(lambda x: x.lower())
        #converting obj do timestamp
        df.timestamp = pd.to_datetime(df.timestamp)

        return trata_dados(df)
    except Exception as e:
        print(e)

In [12]:
df = carrega_dados('train_tratado_outliers_2.csv')

In [26]:
df.head()

,origin_call,origin_stand,taxi_id,dist_perc,start_lat,start_lon,stop_lat,stop_lon,month,day,hour,fds_fer,call_type_a,call_type_b,call_type_c
0,-1.0,-1.0,20000589,2.23,41.141412,-8.618643,41.154489,-8.630838,7,1,0,0,0.0,0.0,1.0
1,-1.0,7.0,20000596,3.46,41.159826,-8.639847,41.170671,-8.665740,7,1,0,0,0.0,1.0,0.0
2,-1.0,-1.0,20000320,17.63,41.140359,-8.612964,41.140530,-8.615970,7,1,0,0,0.0,0.0,1.0
3,-1.0,-1.0,20000520,7.97,41.151951,-8.574678,41.142915,-8.607996,7,1,0,0,0.0,0.0,1.0
4,-1.0,-1.0,20000337,4.81,41.180490,-8.645994,41.178087,-8.687268,7,1,0,0,0.0,0.0,1.0


## Test some algorithms

Testing some algorithms performance (speed, precision, etc)...

In [132]:
# Runs some algorithms for compare

def algs_test(X1, X2, y1, y2):

    algs =[('linear', LinearRegression()),
           ('ridge', Ridge(random_state=99)),
           ('lasso', Lasso(random_state=99)), #lasso 142.7065081365015 561.2795889474457
           ('lsvr', LinearSVR(random_state=99)),
           ('KNN reg', KNeighborsRegressor()), #KNN reg 3.3861972620770264 3.5182287757352873
           ('gboost', GradientBoostingRegressor(random_state=99)),
           ('ada boost', AdaBoostRegressor(random_state=99)),
           ('rnd forest', RandomForestRegressor(random_state=99)),
           ('xgboost', XGBRegressor(random_state=99, n_jobs=-1))
           ]

    for name, alg in algs:
        try:
            alg.fit(X1, y1)
            preds = alg.predict(X2)

            concat = np.apply_along_axis(np.radians, 1, np.concatenate((np.array(y2), preds), axis=1)) 
            hav_dist = np.apply_along_axis(lambda x: 6371*(2*np.arcsin(np.sqrt((np.sin((x[3]-x[1])/2)**2 + np.cos(x[1])*np.cos(x[3])*np.sin((x[2]-x[0])/2)**2)))), 1, concat)

            print('normal',name, hav_dist.mean(), hav_dist.std())
        except: # if algorithm dont have native support to multioutput
            mo = MultiOutputRegressor(alg, n_jobs=-1).fit(X1, y1)
            preds = mo.predict(X2)

            concat = np.apply_along_axis(np.radians, 1, np.concatenate((np.array(y2), preds), axis=1)) 
            hav_dist = np.apply_along_axis(lambda x: 6371*(2*np.arcsin(np.sqrt((np.sin((x[3]-x[1])/2)**2 + np.cos(x[1])*np.cos(x[3])*np.sin((x[2]-x[0])/2)**2)))), 1, concat)

            print('multi',name, hav_dist.mean(), hav_dist.std())

In [133]:
# (excelent) outliers reference: http://colingorrie.github.io/outlier-detection.html

def outliers_iqr(ys):
    quartile_1, quartile_3 = np.percentile(ys, [25, 75])
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * 1.5)
    upper_bound = quartile_3 + (iqr * 1.5)
    return np.where((ys > upper_bound) | (ys < lower_bound))

def outliers_z_score(ys):
    threshold = 3

    mean_y = np.mean(ys)
    print(mean_y)
    stdev_y = np.std(ys)
    z_scores = [(y - mean_y) / stdev_y for y in ys]
    return np.where(np.abs(z_scores) > threshold)


def outliers_modified_z_score(ys):
    threshold = 3.5

    median_y = np.median(ys)
    median_absolute_deviation_y = np.median([np.abs(y - median_y) for y in ys])
    modified_z_scores = [0.6745 * (y - median_y) / median_absolute_deviation_y
                         for y in ys]
    return np.where(np.abs(modified_z_scores) > threshold)

#------------#
# Train and test stay in same scale, because this, 
# scaler.transform is invoked in final's method only.

def train_test_mimax(df, remove_outliers=''):
    
    y = df[['stop_lon', 'stop_lat']]
    X = df.drop(['stop_lon', 'stop_lat'], axis=1)
    scaler = MinMaxScaler().fit(X)

    #-- split train/test #X1, X2, y1, y2
    X1, X2, y1, y2 = train_test_split(X, y, test_size=0.3, random_state=42)
    
    if(remove_outliers=='iqr'):
        idx = outliers_iqr(X1.dist_perc)[0].tolist() 
        X1 = X1[~X1.index.isin(idx)]
        y1 = y1[~y1.index.isin(idx)]
    
    if(remove_outliers=='zscore'):
        idx = outliers_z_score(X1.dist_perc)[0].tolist() 
        X1 = X1[~X1.index.isin(idx)]
        y1 = y1[~y1.index.isin(idx)]
    
    if(remove_outliers=='zscore_mod'):
        idx = outliers_modified_z_score(X1.dist_perc)[0].tolist() 
        X1 = X1[~X1.index.isin(idx)]
        y1 = y1[~y1.index.isin(idx)]
        
    X1 = scaler.transform(X1)
    X2 = scaler.transform(X2)
    
    return X1, X2, y1, y2

### First test - all values

Testing with outliers

In [135]:
algs_test(*train_test_mimax(carrega_dados('train_tratado_outliers_2.csv', n_rows=100000)))

normal linear 3.242526950006376 4.572296738247557
normal ridge 5.035208578741894 7.074214279575095
normal lasso 36.324779071030704 303.92947601040646
multi lsvr 3.1863218127999597 4.762258288342083
normal KNN reg 3.4755950381935605 6.459272728794591
multi gboost 2.7746041839015096 3.200689795563214
multi ada boost 3.146199186825982 3.406473200539824
normal rnd forest 2.5479666072422242 3.406634550586585
multi xgboost 2.7918836267241756 3.2605204176922067


Exception ignored in: <bound method DMatrix.__del__ of <xgboost.core.DMatrix object at 0x000000CF92D16438>>
Traceback (most recent call last):
  File "C:\Users\a46396\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py", line 482, in __del__
    if self.handle is not None:
AttributeError: 'DMatrix' object has no attribute 'handle'


### Removing outliers  by distance using IQR

In [136]:
algs_test(*train_test_mimax(carrega_dados('train_tratado_outliers_2.csv', n_rows=100000), 'iqr'))

normal linear 3.2425833774315493 4.575463792948822
normal ridge 5.065930261976431 7.113446517186023
normal lasso 36.4648121986655 303.9111462474312
multi lsvr 3.1932358868652537 4.7679315770933
normal KNN reg 3.512267713886255 11.501153300541784
multi gboost 2.7720885351485074 3.192751752980376
multi ada boost 3.178256252983925 3.4851434820065754
normal rnd forest 2.5530415257189283 3.3677081978555594
multi xgboost 2.7990510140260403 3.254266204733062


Exception ignored in: <bound method DMatrix.__del__ of <xgboost.core.DMatrix object at 0x000000CF92CE5FD0>>
Traceback (most recent call last):
  File "C:\Users\a46396\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py", line 482, in __del__
    if self.handle is not None:
AttributeError: 'DMatrix' object has no attribute 'handle'


### Removing outliers by distance using Z-Score

In [137]:
algs_test(*train_test_mimax(carrega_dados('train_tratado_outliers_2.csv', n_rows=100000), 'zscore'))

5.524522999999997
normal linear 3.2436593040954613 4.574459667786016
normal ridge 5.039750150854115 7.079425583180473
normal lasso 36.28555882688959 303.93460837250666
multi lsvr 3.1662636567340985 4.768181989539964
normal KNN reg 3.507742216388956 11.5014849400449
multi gboost 2.7784649464210047 3.215223893740343
multi ada boost 8.09105263027827 6.628493068706677
normal rnd forest 2.5466122160199474 3.3476191777671227
multi xgboost 2.7912804981779225 3.227982029127283


Exception ignored in: <bound method DMatrix.__del__ of <xgboost.core.DMatrix object at 0x000000CF945B3C88>>
Traceback (most recent call last):
  File "C:\Users\a46396\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py", line 482, in __del__
    if self.handle is not None:
AttributeError: 'DMatrix' object has no attribute 'handle'


### Removing outliers by distance using Z-Score modified

In [138]:
algs_test(*train_test_mimax(carrega_dados('train_tratado_outliers_2.csv', n_rows=100000), 'zscore_mod'))

normal linear 3.242174052228836 4.573566803728244
normal ridge 5.061994285006496 7.10780595989326
normal lasso 36.48563984433075 303.90842226180456
multi lsvr 3.1965189978395223 4.778043385957267
normal KNN reg 3.5118850508358066 11.501055786030161
multi gboost 2.7727059251229975 3.2011773458718835
multi ada boost 8.673090286200152 5.762570196449435
normal rnd forest 2.558399525354379 3.374796435729142
multi xgboost 2.8103219355324685 3.2733977711563074


Exception ignored in: <bound method DMatrix.__del__ of <xgboost.core.DMatrix object at 0x000000CFCC98C940>>
Traceback (most recent call last):
  File "C:\Users\a46396\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py", line 482, in __del__
    if self.handle is not None:
AttributeError: 'DMatrix' object has no attribute 'handle'


Removing outliers and training multiple algorithms with default parameters did not make much difference.

To have a greater variance, we will use the outliers and the ensemble algorithms (Gradient boost and Random Forest):

## GridSearchCV with Random Forest and Gradient Boost

Testing over 100k lines...

In [216]:
"""
Haversine is the distance between 2 points in earth.
This function is in sklearn.make_scorer format.
"""

# lon1, lat1, lon2, lat2
#  0     1     2     3
def haversine_scorer(y_real, y_pred):
    concat = np.apply_along_axis(np.radians, 1, np.concatenate((np.array(y_real), y_pred), axis=1)) 
    return np.apply_along_axis(lambda x: 6371*(2*np.arcsin(np.sqrt((np.sin((x[3]-x[1])/2)**2 + np.cos(x[1])*np.cos(x[3])*np.sin((x[2]-x[0])/2)**2)))), 1, concat).mean()

In [163]:
score_ = make_scorer(haversine_scorer, greater_is_better=False)

In [39]:
y = df[['stop_lon', 'stop_lat']]
X = df.drop(['stop_lon', 'stop_lat'], axis=1)
X = MinMaxScaler().fit_transform(X)

### Random forest

In [32]:
"""
n_jobs=-1 freeze the screen. Start e thereafter CPU processing is down do 0%
I found this: https://github.com/scikit-learn/scikit-learn/issues/2889
There seems to be no solution!

Because this, n_jobs is auto (1). Take loooong time to train data.
"""
param_grid = {'max_depth':[10, 40],
              'min_samples_leaf':[50, 150]}

gs = GridSearchCV(RandomForestRegressor(n_estimators=100, random_state=99), param_grid=param_grid, cv=10, scoring=score_, verbose=2)

gs.fit(X,y)

Fitting 10 folds for each of 4 candidates, totalling 40 fits
[CV] max_depth=10, min_samples_leaf=50 ...............................
[CV] ................ max_depth=10, min_samples_leaf=50, total=  41.7s
[CV] max_depth=10, min_samples_leaf=50 ...............................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   46.4s remaining:    0.0s


[CV] ................ max_depth=10, min_samples_leaf=50, total=  41.3s
[CV] max_depth=10, min_samples_leaf=50 ...............................
[CV] ................ max_depth=10, min_samples_leaf=50, total=  41.3s
[CV] max_depth=10, min_samples_leaf=50 ...............................
[CV] ................ max_depth=10, min_samples_leaf=50, total=  41.2s
[CV] max_depth=10, min_samples_leaf=50 ...............................
[CV] ................ max_depth=10, min_samples_leaf=50, total=  41.2s
[CV] max_depth=10, min_samples_leaf=50 ...............................
[CV] ................ max_depth=10, min_samples_leaf=50, total=  42.1s
[CV] max_depth=10, min_samples_leaf=50 ...............................
[CV] ................ max_depth=10, min_samples_leaf=50, total=  41.8s
[CV] max_depth=10, min_samples_leaf=50 ...............................
[CV] ................ max_depth=10, min_samples_leaf=50, total=  41.6s
[CV] max_depth=10, min_samples_leaf=50 ...............................
[CV] .

[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed: 33.7min finished


GridSearchCV(cv=10, error_score='raise',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
           oob_score=False, random_state=99, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'max_depth': [10, 40], 'min_samples_leaf': [50, 150]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=make_scorer(haversine_scorer, greater_is_better=False),
       verbose=2)

In [35]:
gs.best_score_

-2.9978036640710832

In [37]:
gs.best_params_

{'max_depth': 40, 'min_samples_leaf': 50}

### Gradient boost

In [48]:
param_grid = {'estimator__min_samples_split':[10, 50],
              'estimator__min_samples_leaf':[50, 150]}

gb = GradientBoostingRegressor(random_state=99, learning_rate=0.1, subsample=0.8, max_depth=8)

gs = GridSearchCV(MultiOutputRegressor(gb), param_grid=param_grid, cv=10, scoring=score_, verbose=2)

gs.fit(X,y)

Fitting 10 folds for each of 4 candidates, totalling 40 fits
[CV] estimator__min_samples_leaf=50, estimator__min_samples_split=10 .
[CV]  estimator__min_samples_leaf=50, estimator__min_samples_split=10, total=  53.2s
[CV] estimator__min_samples_leaf=50, estimator__min_samples_split=10 .


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   58.0s remaining:    0.0s


[CV]  estimator__min_samples_leaf=50, estimator__min_samples_split=10, total=  52.9s
[CV] estimator__min_samples_leaf=50, estimator__min_samples_split=10 .
[CV]  estimator__min_samples_leaf=50, estimator__min_samples_split=10, total=  54.3s
[CV] estimator__min_samples_leaf=50, estimator__min_samples_split=10 .
[CV]  estimator__min_samples_leaf=50, estimator__min_samples_split=10, total=  51.5s
[CV] estimator__min_samples_leaf=50, estimator__min_samples_split=10 .
[CV]  estimator__min_samples_leaf=50, estimator__min_samples_split=10, total=  51.5s
[CV] estimator__min_samples_leaf=50, estimator__min_samples_split=10 .
[CV]  estimator__min_samples_leaf=50, estimator__min_samples_split=10, total=  51.5s
[CV] estimator__min_samples_leaf=50, estimator__min_samples_split=10 .
[CV]  estimator__min_samples_leaf=50, estimator__min_samples_split=10, total=  53.0s
[CV] estimator__min_samples_leaf=50, estimator__min_samples_split=10 .
[CV]  estimator__min_samples_leaf=50, estimator__min_samples_spl

[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed: 36.1min finished


GridSearchCV(cv=10, error_score='raise',
       estimator=MultiOutputRegressor(estimator=GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=8, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples...to', random_state=99,
             subsample=0.8, verbose=0, warm_start=False),
           n_jobs=1),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'estimator__min_samples_split': [10, 50], 'estimator__min_samples_leaf': [50, 150]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=make_scorer(haversine_scorer, greater_is_better=False),
       verbose=2)

In [54]:
gs.best_score_

-3.063807160105668

In [51]:
gs.best_params_

{'estimator__min_samples_leaf': 150, 'estimator__min_samples_split': 10}

## Train all data with Random forest

In [139]:
# load data
df = carrega_dados('train_tratado_outliers_2.csv')

In [140]:
df.head(2)

,origin_call,origin_stand,taxi_id,dist_perc,start_lat,start_lon,stop_lat,stop_lon,month,day,hour,fds_fer,call_type_a,call_type_b,call_type_c
0,-1.0,-1.0,20000589,2.23,41.141412,-8.618643,41.154489,-8.630838,7,1,0,0,0.0,0.0,1.0
1,-1.0,7.0,20000596,3.46,41.159826,-8.639847,41.170671,-8.665740,7,1,0,0,0.0,1.0,0.0


In [141]:
# separating
y = df[['stop_lon', 'stop_lat']]
X = df.drop(['stop_lon', 'stop_lat'], axis=1)

In [142]:
# using Min max scaler. Fit first then transform it after.

# create scaler
scaler = MinMaxScaler().fit(X)
#transforming
X = scaler.transform(X)

In [143]:
# fit
# n_jobs=-1 works fine here. Seems that screen freeze have relation with custom score and how gridsearch use that.

reg = RandomForestRegressor(n_estimators=100, n_jobs=-1, max_depth=8, min_samples_leaf=50, random_state=99, verbose=2)
reg.fit(X,y)

building tree 1 of 100
building tree 2 of 100
building tree 3 of 100building tree 4 of 100

building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100


[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  2.5min


building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54 of 100
building tree 55 of 100
building tree 56 of 100
building tree 57 of 100
building tree 58 of 100
building tree 59 of 100
building tree 60 of 100
building tree 61 of 100
building tree 62 of 100
building tree 63 of 100
building tree 64 of 100
building tree 65 of 100
building tree 66 of 100
building tree 67 of 100
building tree 68 of 100
building tree 69 of 100
building tree 70 of 100
building tree 71 of 100
building tree 72 of 100
building tree 73 of 100
building tree 74 of 100
building tree 75 of 100
building tree 76 of 100
building tree 77 of 100
building tree 78 of 100
building tree 79

[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  7.0min finished


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=8,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=50, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
           oob_score=False, random_state=99, verbose=2, warm_start=False)

In [145]:
# saving pickle

name = r'pickles/'+str(datetime.datetime.now().date())+'_random_forest_0.3_.save'
pickle.dump(reg, open(name,'wb'))

In [59]:
# Loading model

#name_load = r'pickles/2018-09-28_random_forest_0.3_.save'
#reg = pickle.load(open(name_load, 'rb'))

### Test

Each one of These datasets refer to trips occurred between 01/07/2014 and 31/12/2014. Each one of these data sets will provide a snapshot of the current network status on a given timestamp. It will provide partial trajectories for each one of the on-going trips during that specific moment.

The five snapshots included on the test set refer to the following timestamps:

- 14/08/2014 18:00:00
- 30/09/2014 08:30:00
- 06/10/2014 17:45:00
- 01/11/2014 04:00:00
- 21/12/2014 14:30:00

In [197]:
df_test = carrega_dados('test_trated.csv', None, True)

In [198]:
df_test.head()

,trip_id,origin_call,origin_stand,taxi_id,dist_perc,start_lat,start_lon,stop_lat,stop_lon,month,day,hour,fds_fer,call_type_a,call_type_b,call_type_c
0,T1,-1.0,15.0,20000542,0.51,41.148522,-8.585676,41.146623,-8.584884,8,14,17,0,0.0,1.0,0.0
1,T2,-1.0,57.0,20000108,2.28,41.145570,-8.610876,41.163597,-8.601894,8,14,17,0,0.0,1.0,0.0
2,T3,-1.0,15.0,20000370,3.91,41.148558,-8.585739,41.167719,-8.574903,8,14,17,0,0.0,1.0,0.0
3,T4,-1.0,53.0,20000492,0.18,41.141169,-8.613963,41.140980,-8.614638,8,14,17,0,0.0,1.0,0.0
4,T5,-1.0,18.0,20000621,0.00,41.148036,-8.619903,41.148036,-8.619894,8,14,17,0,0.0,1.0,0.0


In [199]:
df_test.set_index('trip_id', inplace=True)
y2 = df_test[['stop_lon', 'stop_lat']]
X2 = df_test.drop(['stop_lon', 'stop_lat'], axis=1)

In [200]:
X2_scaled = scaler.transform(X2)

In [201]:
preds = reg.predict(X2_scaled)

[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished


In [202]:
score_(reg, X2_scaled, y2)

[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished


-2.558431034608694

The model fulfilled its role and put itself well with new data achieving to generalize new points with good performance.

## Ploting results

In [193]:
# Essa é a mesma formula usada para fazer o scorer.
# Aqui esta mais legível e computa apenas a distancia
# entre os pontos.

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    a = sin((lat2 - lat1)/2)**2 + cos(lat1) * cos(lat2) * sin((lon2 - lon1)/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

In [217]:
result = y2.copy()
result['pred_stop_lat'] = preds[:,0]
result['pred_stop_lon'] = preds[:,1]
result['diff_dist'] = result.apply(lambda x: haversine(x[0], x[1], x[2], x[3]), axis=1)

In [218]:
result = pd.concat([result, X2[['start_lon', 'start_lat']]], axis=1)

In [219]:
result.head()

,stop_lon,stop_lat,pred_stop_lat,pred_stop_lon,diff_dist,start_lon,start_lat
trip_id,,,,,,,
T1,-8.584884,41.146623,-8.604468,41.156994,2.004555,-8.585676,41.148522
T2,-8.601894,41.163597,-8.612704,41.153483,1.443557,-8.610876,41.145570
T3,-8.574903,41.167719,-8.604984,41.157335,2.770293,-8.585739,41.148558
T4,-8.614638,41.140980,-8.612704,41.153483,1.399627,-8.613963,41.141169
T5,-8.619894,41.148036,-8.621989,41.155700,0.870028,-8.619903,41.148036
